In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
#gather all fine tuning logs
file_paths = ['Document_Classification/logs/doc_class_stats.txt',
             'NER/logs/NER_stats.txt',
             'QA/logs/QA_stats.txt',
             'QA_BioASQ/logs/QA_BIO_stats.txt',
            ]
    

In [6]:
results = {}

for path in file_paths:
    label = path.split('/')[0]
    results[label] = []
    with open(path) as f:
        for line in f.readlines():
            output = json.loads(line)
            temp = {'model_name': output['model_name'], 'metric': output['metric'], 'score':output['mean_score']}
            results[label].append(temp)

In [7]:
for key in results:
    print(len(results[key]), key)

12 Document_Classification
10 NER
10 QA
10 QA_BioASQ


In [197]:
doc_class = results['Document_Classification'][:10]

In [8]:
frames = []
for key in results:
    df = pd.DataFrame(results[key])
    frames.append(df)


In [10]:
frames[0]

,model_name,metric,score
0,bert-base-uncased,f1-score,0.7898
1,model-trained-0-3531-4GB,f1-score,0.7858
2,model-trained-18-67089-4GB,f1-score,0.8114
3,model-trained-36-130647-4GB,f1-score,0.8124
4,run_8GB_model-trained-0-7063,f1-score (micro),0.8051
5,run_8GB_model-trained-8-63567,f1-score (micro),0.8250
6,run_8GB_model-trained-22-162449,f1-score (micro),0.8293
7,model-trained-0-10596-12GB,f1-score,0.8111
8,model-trained-3-42384-12GB,f1-score,0.8143
9,model-trained-5-63576-12GB,f1-score,0.8246


In [199]:
frames[0] = frames[0].loc[0:9,:]

In [200]:
pretraining_corpus_size = ['20GB', '4GB', '4GB', '4GB', '8GB', '8GB', '8GB', '12GB', '12GB', '12GB']
pretraining_steps = [1000000, 3500, 67000, 130000, 7063, 63500, 162449, 10500, 42000, 63500]

In [201]:
dc = pd.concat((frames[0], pd.Series(pretraining_corpus_size)), axis=1)

In [202]:
dc.rename(columns={0:'corpus_size'}, inplace=True)

In [203]:
dc = pd.concat((dc, pd.Series(pretraining_steps)), axis=1)
dc.rename(columns={0:'pretraining_steps'}, inplace=True)

In [204]:
model_names_8GB = ['model-trained-0-7063-8GB', 'model-trained-8-63567-8GB',
       'model-trained-22-162449-8GB']

In [205]:
for i, x in enumerate(range(4,7)):
    dc.loc[x,'model_name'] = model_names_8GB[i]

In [209]:
dc.to_csv('doc_class_results.csv', index=False)

In [208]:
dc.sort_values('score', ascending=False)

,model_name,metric,score,corpus_size,pretraining_steps
6,model-trained-22-162449-8GB,f1-score (micro),0.8293,8GB,162449
5,model-trained-8-63567-8GB,f1-score (micro),0.8250,8GB,63500
9,model-trained-5-63576-12GB,f1-score (micro),0.8246,12GB,63500
8,model-trained-3-42384-12GB,f1-score (micro),0.8143,12GB,42000
3,model-trained-36-130647-4GB,f1-score (micro),0.8124,4GB,130000
2,model-trained-18-67089-4GB,f1-score (micro),0.8114,4GB,67000
7,model-trained-0-10596-12GB,f1-score (micro),0.8111,12GB,10500
4,model-trained-0-7063-8GB,f1-score (micro),0.8051,8GB,7063
0,bert-base-uncased,f1-score (micro),0.7898,20GB,1000000
1,model-trained-0-3531-4GB,f1-score (micro),0.7858,4GB,3500


In [207]:
dc['metric'] = 'f1-score (micro)'